# Simple `openrouteservice-py` examples

This notebook is just a small reference on how to use `openrouteservice` package to request:
- [directions](https://openrouteservice.org/dev)
- [isochrones](https://openrouteservice.org/dev/#/api-docs/v2/isochrones/{profile}/post)
- [matrix](https://openrouteservice.org/dev/#/api-docs/v2/matrix/{profile}/post)

In [1]:
import openrouteservice
import folium

client = openrouteservice.Client(base_url="http://localhost:8080/ors")

## TOC

- **[Directions](#Directions)**
- **[Isochrones](#Isochrones)**
- **[Matrix](#Matrix)**


## Directions

In [2]:
# Some coordinates in NYC
coordinates = [[-73.97875, 40.72208], [-73.98631, 40.72131]]


m = folium.Map(location=list(reversed(coordinates[0])),tiles='cartodbpositron', zoom_start=13)

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    options={"avoid_features": ["steps"]},
    validate=False,
)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    

m

## Isochrones

In [3]:
# Some coordinate in NYC
coordinate = [(-73.97875, 40.72208)]

m = folium.Map(location=list(reversed(coordinate[0])), tiles='cartodbpositron', zoom_start=13)

iso = client.isochrones(
    locations=coordinate,
    profile='foot-walking',
    range=[900]
)


folium.Marker(location=list(reversed(coordinate[0])), popup='Location').add_to(m)

for isochrone in iso['features']:
    folium.Polygon(locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
                  fill='00ff00',
                  opacity=0.5).add_to(m)
    
m

## Matrix

In [4]:
# Some coordinates in NYC
coordinates = [[ -74.00782277832863,40.71299873734118], [ -74.00552620175422, 40.71342967094514], [ -74.00360397186132, 40.71313254481554], [-74.00507241847856, 40.711027500941185]]


m = folium.Map(location=list(reversed(coordinates[0])), tiles='cartodbpositron', zoom_start=15)


matrix = client.distance_matrix(
    locations=coordinates,
    profile='foot-walking',
    metrics=['distance', 'duration'],
    validate=False,
    units = 'mi'
)

for marker in coordinates:
    folium.Marker(location=list(reversed(marker))).add_to(m)

print("Duration Matrix:")
for row in matrix['durations']:
    print(row)

print("\nDistance Matrix:")
for row in matrix['distances']:
    print(row)

m

Duration Matrix:
[0.0, 206.09, 332.89, 307.85]
[206.09, 0.0, 151.26, 335.85]
[332.89, 151.26, 0.0, 301.01]
[307.85, 335.85, 301.01, 0.0]

Distance Matrix:
[0.0, 0.18, 0.29, 0.27]
[0.18, 0.0, 0.13, 0.29]
[0.29, 0.13, 0.0, 0.26]
[0.27, 0.29, 0.26, 0.0]
